In [28]:
import os
import pickle 
import sys
import pandas as pd
from collections import Counter
from Bio import SeqIO


# if yeast_esr_flag =='Y':
yeast_esr_exp_path = os.path.normpath('C:/Users/heineib/Documents/GitHub/yeast_esr_expression_analysis') + os.sep
#io_library_path_core = io_library_path + 'core' + os.sep
if not(yeast_esr_exp_path in sys.path):
    sys.path.append(yeast_esr_exp_path)
    print("Added " + yeast_esr_exp_path + " to path" )

import yeast_esr_exp
yeast_esr_exp.base_dir = yeast_esr_exp_path
yeast_esr_exp.data_processing_dir = yeast_esr_exp_path + os.path.normpath('expression_data') + os.sep


y1000plus_tools_flag = input("Use y1000plus_tools functions? (Y/n)")

# if y1000plus_tools_flag =='Y':
y1000plus_tools_path = os.path.normpath('C:/Users/heineib/Documents/GitHub/y1000plus_tools') + os.sep
#io_library_path_core = io_library_path + 'core' + os.sep
if not(y1000plus_tools_path in sys.path):
    sys.path.append(y1000plus_tools_path)
    print("Added " + y1000plus_tools_path + " to path" )

import y1000plus_tools
y1000plus_tools.base_dir = os.path.normpath('C:/Users/heineib/Documents/GitHub/y1000plus_tools') + os.sep
print("y1000plus_tools dir is " + y1000plus_tools.base_dir )                      
y1000plus_tools.y1000plus_dir = y1000plus_tools.base_dir + 'data' + os.sep 
print("y1000plus data dir is " + y1000plus_tools.y1000plus_dir)


    

y1000plus_dir = os.path.normpath('C:/Users/heineib/Documents/GitHub/y1000plus_tools/data') + os.sep
#base_dir = os.path.normpath('G:/My Drive/Crick_LMS/projects/diverse_yeasts')


#Load y1000 species table
y1000_species = pd.read_csv(y1000plus_dir + os.path.normpath("y1000plus_tools_data/y1000plus/y1000_species_table.csv"), index_col=0)


#D. hansenii and H. osmophilia  missing from 220811_Diverse_species_SF08_6h_CN1_woSlowGrowth.csv?
spec_abbrev_map = {'BY': 'saccharomyces_cerevisiae', 
                   'SC': 'saccharomyces_cerevisiae', 
                   'CA': 'candida_albicans', 
                   'CT': 'candida_tropicalis', 
                   'KL': 'kluyveromyces_lactis', 
                   'KM': 'kluyveromyces_marxianus', 
                   'LT': 'lachancea_thermotolerans',
                   'PP': 'komagataella_pastoris', 
                   'WA': 'wickerhamomyces_anomalus', 
                   'ZR': 'zygosaccharomyces_rouxii'
                  }


##Need to add something for S. cer
shen_proteome_dir = os.path.normpath(y1000plus_dir + 'shen_2018_data/0_332yeast_genomes/332_genome_annotations/pep') + os.sep 
uniprot_proteome_dir = os.path.normpath('G:/My Drive/Crick_LMS/external_data/genomes/diverse_strains/proteomes') + os.sep 

#(species genome name: (shen_fname, uniprot_fname)): 
proteome_fasta_dict = {'kluyveromyces_lactis': {'shen': shen_proteome_dir + 'kluyveromyces_lactis.max.pep', 
                                                'uniprot': uniprot_proteome_dir + 'Klac_uniprot-proteome_UP000000598.fasta'
                                               }, 
                       'wickerhamomyces_anomalus': {'shen': shen_proteome_dir + 'wickerhamomyces_anomalus.max.pep',
                            'uniprot': uniprot_proteome_dir + 'Wano_uniprot-proteome_UP000094112.fasta'
                           }, 
                       'lachancea_thermotolerans': {'shen': shen_proteome_dir + 'lachancea_thermotolerans.max.pep',
                            'uniprot': uniprot_proteome_dir + 'Lthe_uniprot-proteome_UP000002036.fasta'
                           }, 
                       'komagataella_pastoris': {'shen': shen_proteome_dir + 'komagataella_pastoris.max.pep',
                            'uniprot': uniprot_proteome_dir + 'Kpas_uniprot-proteome_UP000000314.fasta'
                           }, 
                        'candida_tropicalis': {'shen': shen_proteome_dir + 'candida_tropicalis.max.pep',
                            'uniprot': uniprot_proteome_dir + 'Ctro_uniprot-proteome_UP000002037.fasta'
                           },
                        'debaryomyces_hansenii': {'shen': shen_proteome_dir + 'debaryomyces_hansenii.max.pep',
                            'uniprot': uniprot_proteome_dir + 'Dhan_uniprot-proteome_UP000000599.fasta'
                           }, 
                        'yHMPu5000034957_hanseniaspora_osmophila_160519': {'shen': shen_proteome_dir + 'yHMPu5000034957_hanseniaspora_osmophila_160519.max.pep',
                            'uniprot': uniprot_proteome_dir + 'Hosm_uniprot-proteome_UP000095728.fasta'
                           }, 
                        'kluyveromyces_marxianus': {'shen': shen_proteome_dir + 'kluyveromyces_marxianus.max.pep',
                            'uniprot': uniprot_proteome_dir + 'Kmar_uniprot-proteome_UP000065495.fasta'
                           }, 
                        'zygosaccharomyces_rouxii': {'shen': shen_proteome_dir + 'zygosaccharomyces_rouxii.max.pep',
                            'uniprot': uniprot_proteome_dir + 'Zrou_uniprot-proteome_UP000008536.fasta'
                           }, 
                        'saccharomyces_cerevisiae': {'uniprot': uniprot_proteome_dir + 'Scer_uniprot-proteome_UP000002311.fasta'}
                      }



base_dir = os.path.normpath('G:/My Drive/Crick_LMS/projects/diverse_yeasts')

pairwise_scores_fname = base_dir + os.sep + 'alphafold' + os.sep + 'pairwise_scores.pkl'
# Load pairwise_scores
with open(pairwise_scores_fname, 'rb') as f: 
    pairwise_scores = pickle.load(f)

    
#Load Dataset
combined_data = pd.read_csv(base_dir + os.sep + os.path.normpath('proteomics/220811/220811_Diverse_species_SF08_6h_CN1_woSlowGrowth.csv'), index_col=0)




#spec_orig_genome_lookup = dict(zip(y1000_species['spec_og_id'], y1000_species['original_genome_id']))


# #Load S.Cer lookup table: 
# scer_lookup_fname = y1000plus_dir + os.path.normpath('y1000plus_tools_data/y1000plus/id_lookups/saccharomyces_cerevisiae.csv')

# scer_lookup = pd.read_csv(scer_lookup_fname, index_col=0)
# scer_y1000_id_gene_id_lookup = dict(zip(scer_lookup['y1000_id'], scer_lookup.index))







Use y1000plus_tools functions? (Y/n) Y


y1000plus_tools dir is C:\Users\heineib\Documents\GitHub\y1000plus_tools\
y1000plus data dir is C:\Users\heineib\Documents\GitHub\y1000plus_tools\data\


In [57]:
#Save mapping files for each species in the diverse yeast proteomics dataset.  Maps from 
#Also saves log file for numbers of ids found and lists multimappers. 

fname_log = base_dir + os.sep + os.path.normpath('proteomics/220811/uniprot_2_y1000id_log.txt')

with open(fname_log,'w') as f_log: 
    for spec_abbrev, spec in spec_abbrev_map.items():
        #Skipping S. cer because I don't have pairwise scores for that
        if not(spec_abbrev in {'SC', 'BY', 'CA'}): 
               #spec_abbrev = 'KL' 
            #spec = spec_abbrev_map[spec_abbrev]
            fname_out = base_dir + os.sep + os.path.normpath('proteomics/220811/uniprot_2_y1000id_' + spec_abbrev + '.csv')


            #Load map from full name to shen identifier 

            spec_lookup_fname = y1000plus_dir +  os.path.normpath('y1000plus_tools_data/y1000plus/id_lookups/' + spec + '.csv')
            spec_lookup = pd.read_csv(spec_lookup_fname, index_col=0)

            shen_full_2_y1000_id = dict(zip(spec_lookup['gene_full'], spec_lookup['y1000_id']))

            #Make Dictionary from pairwise scores
            pairwise_scores_spec = pairwise_scores[spec]

            sim_score_high_thresh = 75
            sim_score_low_thresh = 60
            sim_score_diff_thresh = 8

            shen_full_2_uniprot = {}

            f_log.write('Species: ' + spec + '\n')

            for shen_full, scores in pairwise_scores_spec.items(): 

                #shen_full = 'augustus_masked-CR382121-processed-gene-0.43'
                #scores = pairwise_scores_spec[shen_full]

                scores_sorted = scores.sort_values(ascending=False)
                maxval = scores_sorted[0]
                diff_top2_val = maxval - scores_sorted[1]

                if ((maxval>sim_score_high_thresh) | ((maxval>sim_score_low_thresh) & (diff_top2_val>sim_score_diff_thresh))): 
                    uniprot_id = scores_sorted.index[0].split('|')[1]  #The id looks like tr|<ID>|<name_specabbrev>, so we only extract the middle id

                else: 
                    f_log.write('No clear uniprot match: ' + shen_full)
                    uniprot_id = None

                shen_full_2_uniprot[shen_full] = uniprot_id



            f_log.write('Shen IDs in selected OGs: ' + str(len(pairwise_scores_spec)) + '\n')
            shen_mapped = [value for value in shen_full_2_uniprot.values() if value != None]
            f_log.write('Shen IDs mapped to uniprot: ' + str(len(shen_mapped)))
            if len(shen_mapped)==len(set(shen_mapped)):
                f_log.write('One shen ID mapped to multiple Uniprot IDs\n')
                f_log.write('Duplicate Uniprot IDs: \n')
                for name, count in Counter(shen_mapped).items(): 
                    if count>1: 
                        f_log.write(name + '\n')

            #check how many shen_ids are missing
            f_log.write('total Shen IDs: ' + str(len(spec_lookup))+ '\n')
            assert len(set(shen_full_2_uniprot.keys()) - set(spec_lookup['gene_full']))==0, 'Some unidentifed shen ids'
            f_log.write('Shen IDs not mapped from og score matrix: ' + str(len(set(spec_lookup['gene_full']) - set(shen_full_2_uniprot.keys())))+ '\n')

            #Check how many uniprot ids are missing
            uniprot_ids = []
            uniprot_fasta = SeqIO.parse(proteome_fasta_dict[spec]['uniprot'], "fasta")
            for seq in uniprot_fasta: 
                name = seq.name
                uniprot_ids.append(name.split('|')[1])

            f_log.write('Total Uniprot IDs: ' +  str(len(uniprot_ids))+ '\n')
            assert len(set(shen_full_2_uniprot.values()) - (set(uniprot_ids) | {None}))==0, 'Some unidentifed Uniprot IDs'
            f_log.write('Uniprot IDs not mapped from og score matrix: ' + str(len(set(uniprot_ids) - set(shen_full_2_uniprot.values())))+ '\n')

            #Check how many uniprot ids are missing from Federica's data (assert none)
            data_uniprot_ids = combined_data.loc[combined_data['Species']==spec_abbrev, :]['Protein.Ids'] 
            f_log.write('Uniprot IDs in data: ' + str(len(set(data_uniprot_ids)))+ '\n')

            #Check how many from federica's data set map to shen ids
            f_log.write('Uniprot IDs in data that map to OG proteins: ' + str(len(set(data_uniprot_ids) & set(shen_full_2_uniprot.values())  ))+ '\n')





            #Make dictionary of Uniprot id to shen identifer 

            uniprot_id_2_y1000_id_spec = {}

            with open(fname_out, 'w') as f_out: 
                for key, value in shen_full_2_uniprot.items(): 
                    if value != None: 
                        uniprot_id_2_y1000_id_spec[value] = shen_full_2_y1000_id[key]
                        f_out.write(value + ', ' + shen_full_2_y1000_id[key] +'\n')